In [0]:
#from google.colab import drive
#drive.mount('/content/drive')
#! rm -r "/content/drive/My Drive/Minecraft/images/"
! mkdir "/content/drive/My Drive/Minecraft/images"

In [6]:
! pip install tensorflow-gpu==1.15 keras==2.1.5

     |████████████████████████████████| 411.5MB 45kB/s 
     |████████████████████████████████| 337kB 38.1MB/s 
     |████████████████████████████████| 3.8MB 37.7MB/s 
     |████████████████████████████████| 512kB 40.6MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=e97c51e5796b67df5b93eeede9567b1965369bdac713b41cc296c82b5aeafc5e
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.2.0 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement tensorboard<2.3.0,>=2.2.0, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement tensorflow-estimator<2.3.0,>=2.2.0, but you'll have tensorflow-estimator 1.15.1 which is incompatible.
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found exis

In [7]:
from __future__ import print_function, division
import keras
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D, ZeroPadding3D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D, UpSampling3D, Conv3D
from keras.models import Sequential, Model
from keras.optimizers import Adam, SGD
import os
from tensorboardcolab import *
tbc=TensorBoardColab()
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0' 
import matplotlib.pyplot as plt
import tensorflow as tf
print(tf.test.gpu_device_name())
import sys
import datetime
import numpy as np

with np.load("drive/My Drive/Minecraft/largeNorm.npz") as data:
    images = data['a']
print(tf.__version__)



Using TensorFlow backend.


Wait for 8 seconds...
TensorBoard link:
https://61744e9bc803.ngrok.io
/device:GPU:0
1.15.0


In [0]:
class DCGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 256
        self.img_cols = 16
        self.img_depth = 16
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.img_depth, self.channels)
        self.latent_dim = 100

        optimizerD = Adam(0.0004, 0.5)
        optimizerG = Adam(0.0001, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizerD,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizerG)

    def write_log(self, name, value, epoch):
        tbc.save_value("Loss", name, epoch, value)
        tbc.flush_line(name)
        tbc.close()

    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 4 * 4 * 64, activation="linear", input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Reshape((64, 4, 4, 128)))
        model.add(UpSampling3D())
        model.add(Conv3D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(UpSampling3D())
        model.add(Conv3D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Conv3D(1, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv3D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv3D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding3D(padding=((0,1),(0,1), (0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv3D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv3D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=32, save_interval=5):

        # Load the dataset
        X_train = images

        # Rescale -1 to 1
        #print(X_train[0][0])
        #X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=4)
        #print(X_train[0][0])

        # Adversarial ground truths
        valid = np.full((batch_size, 1), 0.9)
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half of images
            epoch += 49000
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            # Sample noise and generate a batch of new images
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator (wants discriminator to mistake images as real)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
            self.write_log("g_loss", g_loss, epoch)
            self.write_log("d_loss", d_loss[0], epoch)
            self.write_log("Accuracy", 100*d_loss[1], epoch)

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)

    def save_imgs(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        #gen_imgs = 0.5 * gen_imgs + 0.5

        np.save("drive/My Drive/Minecraft/images/mnist_%d.npy" % epoch, gen_imgs)
        self.combined.save_weights('drive/My Drive/Minecraft/images/mnist-%d.h5' % epoch)
        '''
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("C:/Users/adamj/Documents/MinecraftTerrainGeneration/DCGAN/images/mnist_%d.png" % epoch)
        plt.close()
        '''


In [0]:
dcgan = DCGAN()
#dcgan.combined.load_weights("/content/drive/My Drive/Minecraft/images/mnist-48900.h5", reshape=True)
dcgan.train(epochs=100000, batch_size=16, save_interval=1000)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 128, 8, 8, 32)     896       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 128, 8, 8, 32)     0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 128, 8, 8, 32)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 64, 4, 4, 64)      55360     
_________________________________________________________________
zero_padding3d_1 (ZeroPaddin (None, 65, 5, 5, 64)      0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 65, 5, 5, 64)      256       
_______________________

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:973: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:973: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


49000 [D loss: 6.608877, acc.: 0.00%] [G loss: 0.535640]




/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:973: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


49001 [D loss: 0.506083, acc.: 50.00%] [G loss: 6.949461]
49002 [D loss: 1.355857, acc.: 15.62%] [G loss: 4.077263]
49003 [D loss: 0.845362, acc.: 31.25%] [G loss: 12.750123]
49004 [D loss: 3.077935, acc.: 0.00%] [G loss: 13.808189]
49005 [D loss: 0.788772, acc.: 43.75%] [G loss: 11.982235]
49006 [D loss: 0.314823, acc.: 50.00%] [G loss: 12.906727]
49007 [D loss: 1.819013, acc.: 0.00%] [G loss: 14.506286]
49008 [D loss: 3.584263, acc.: 50.00%] [G loss: 11.528646]
49009 [D loss: 0.530734, acc.: 50.00%] [G loss: 10.448692]
49010 [D loss: 0.342766, acc.: 50.00%] [G loss: 10.324207]
49011 [D loss: 0.338357, acc.: 50.00%] [G loss: 2.650177]
49012 [D loss: 0.529725, acc.: 50.00%] [G loss: 0.448729]
49013 [D loss: 0.457947, acc.: 50.00%] [G loss: 0.938923]


In [0]:
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()